In [1]:
import numpy as np
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import re
from IterativeLIFModel import *
from SCNNLayer import *
import time

In [2]:
class DVS_Gesture(Dataset):
    
    def __init__(self, dataPath, mode, duration, timestep):
        
        self.mode = mode
        if mode == 'train':
            self.dataPath = dataPath + 'tensor_train/' #../../../DATA/DVS-Gesture/processed/dvs_train/
            self.dataFiles = os.listdir(self.dataPath) #a list
        elif mode == 'test':
            self.dataPath = dataPath + 'tensor_test/' #../../../DATA/DVS-Gesture/processed/dvs_test
            self.dataFiles = os.listdir(self.dataPath)
        else:
            print('typo!')
        #self.duration = duration # 30*1000 #ms->um，每次采样持续30毫秒(30*1000微秒)
        #self.timestep = timestep # 1200 * 1000 #ms->um，整个采样时长
        #self.bins = int(self.timestep/self.duration) # 次，每个动作采样40次
    
    def __getitem__(self, index):
        data_dir = self.dataPath + self.dataFiles[index]
        tensor_data = torch.load(data_dir)
        label_pattern = r'_(\d+)_'
        label = int(re.findall(label_pattern,self.dataFiles[index])[0]) - 1
        label  = torch.tensor(label, dtype=torch.int64)
        return tensor_data, label
    
    def __len__(self):
        return len(self.dataFiles)

In [3]:
dataPath = '../../../DATA/DVS-Gesture/processed/'
duration = 30*1000
timestep = 1200 * 1000

In [4]:
trainSet = DVS_Gesture(dataPath = dataPath, mode = 'train')
trainLoader = DataLoader(dataset = trainSet, batch_size=8, shuffle=True, num_workers = 0)

In [5]:
testSet = DVS_Gesture(dataPath = dataPath, mode = 'test')
testLoader = DataLoader(dataset = testSet, batch_size=8, shuffle=True, num_workers = 0)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
device

device(type='cuda')

In [8]:
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
acc_record = []
num_epochs = 100 # max epoch
num_classes = 11
batch_size = 8
names = 'STBPmodelDVS-Gesture'

In [9]:
learning_rate = 1e-3

In [10]:
snn = SCNN()
snn.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(snn.parameters(), lr=learning_rate)

In [11]:
# Dacay learning_rate
def lr_scheduler(optimizer, epoch, init_lr=0.1, lr_decay_epoch=50):
    """Decay learning rate by a factor of 0.1 every lr_decay_epoch epochs."""
    if epoch % lr_decay_epoch == 0 and epoch > 1:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1
    return optimizer

In [12]:
for epoch in range(num_epochs):
    train_loss = 0
    start_time = time.time()
    for i, (train_images, train_labels) in enumerate(trainLoader):
        snn.zero_grad()
        optimizer.zero_grad()
        
        train_images = train_images.float().to(device)
        train_predicts = snn(train_images)
        train_labels = torch.zeros(train_labels.shape[0], num_classes).scatter_(1, train_labels.view(-1,1), 1.)
                                            #将index视作列（dim = 1），按照train_labels.view(-1,1)值作为index，把src =“1”插入到zeros中
        loss = criterion(train_predicts.cpu(), train_labels)
        train_loss = loss.item() + train_loss
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 25 == 0:#每100个batch报告一次，即100*100张图片报告一次
            print('Epoch [%d/%d], Train Step [%d/%d], Train Loss = %.5f' 
                    %(epoch + 1, num_epochs, i + 1, len(trainSet)//batch_size, train_loss))
            train_loss = 0
            print('Time elasped:', time.time() - start_time)
    correct = 0
    total = 0
    optimizer = lr_scheduler(optimizer, epoch, learning_rate, 40)
    
    with torch.no_grad():
        for j, (test_images, test_targets) in enumerate(testLoader):
            optimizer.zero_grad()
            snn.zero_grad()
            
            test_images = test_images.float().to(device)
            test_predicts = snn(test_images)
            test_labels = torch.zeros(test_targets.shape[0], num_classes).scatter_(1, test_targets.view(-1,1), 1.)
            loss = criterion(test_predicts.cpu(), test_labels)
            
            _, predicted = test_predicts.cpu().max(1)
            total = total + float(test_targets.size(0))
            correct = correct + float(predicted.eq(test_targets).sum().item())
            if (j + 1) % 25 == 0:
                acc = 100. * float(correct)/float(total)
                print('Test Step [%d/%d], Acc: %.5f' %(j + 1, len(testLoader), acc))
    acc = float(100 * correct / total)
    print('Epoch [%d/%d] \t Test Accuracy Over Test Dataset: %.3f' %(epoch + 1, num_epochs, acc))
    acc_record.append(acc)
    if epoch % 5 == 0:
        print('Saving......')
        state = {
            'net': snn.state_dict(),
            'acc': acc,
            'epoch': epoch,
            'acc_record': acc_record,
        }
        if not os.path.isdir('./history'):
            os.mkdir('./history')
        torch.save(state, './history/' + names + '.pk')

Epoch [1/100], Train Step [25/147], Train Loss = 2.25930
Time elasped: 12.095783710479736
Epoch [1/100], Train Step [50/147], Train Loss = 2.19749
Time elasped: 20.498173713684082
Epoch [1/100], Train Step [75/147], Train Loss = 2.18850
Time elasped: 28.99190402030945
Epoch [1/100], Train Step [100/147], Train Loss = 2.18363
Time elasped: 37.446715354919434
Epoch [1/100], Train Step [125/147], Train Loss = 2.18925
Time elasped: 45.87733793258667
Test Step [25/36], Acc: 5.50000
Epoch [1/100] 	 Test Accuracy Over Test Dataset: 8.333
Saving......
Epoch [2/100], Train Step [25/147], Train Loss = 2.17073
Time elasped: 8.328710556030273
Epoch [2/100], Train Step [50/147], Train Loss = 2.17055
Time elasped: 16.836825370788574
Epoch [2/100], Train Step [75/147], Train Loss = 2.17168
Time elasped: 25.277524948120117
Epoch [2/100], Train Step [100/147], Train Loss = 2.17248
Time elasped: 33.74984693527222
Epoch [2/100], Train Step [125/147], Train Loss = 2.17168
Time elasped: 42.19629383087158
T

Epoch [16/100], Train Step [50/147], Train Loss = 2.09537
Time elasped: 18.00267791748047
Epoch [16/100], Train Step [75/147], Train Loss = 2.13904
Time elasped: 27.20475482940674
Epoch [16/100], Train Step [100/147], Train Loss = 2.12901
Time elasped: 35.96785378456116
Epoch [16/100], Train Step [125/147], Train Loss = 2.16869
Time elasped: 44.736727714538574
Test Step [25/36], Acc: 14.50000
Epoch [16/100] 	 Test Accuracy Over Test Dataset: 16.667
Saving......
Epoch [17/100], Train Step [25/147], Train Loss = 2.14802
Time elasped: 9.00916314125061
Epoch [17/100], Train Step [50/147], Train Loss = 2.20647
Time elasped: 18.11125087738037
Epoch [17/100], Train Step [75/147], Train Loss = 2.16003
Time elasped: 27.1418673992157
Epoch [17/100], Train Step [100/147], Train Loss = 2.17898
Time elasped: 36.17314076423645
Epoch [17/100], Train Step [125/147], Train Loss = 2.21685
Time elasped: 45.19124245643616
Test Step [25/36], Acc: 9.50000
Epoch [17/100] 	 Test Accuracy Over Test Dataset: 8.

Epoch [31/100], Train Step [75/147], Train Loss = 2.16793
Time elasped: 25.32012152671814
Epoch [31/100], Train Step [100/147], Train Loss = 2.16344
Time elasped: 33.77110767364502
Epoch [31/100], Train Step [125/147], Train Loss = 2.20825
Time elasped: 42.21078109741211
Test Step [25/36], Acc: 13.00000
Epoch [31/100] 	 Test Accuracy Over Test Dataset: 14.236
Saving......
Epoch [32/100], Train Step [25/147], Train Loss = 2.11138
Time elasped: 8.283886909484863
Epoch [32/100], Train Step [50/147], Train Loss = 2.19888
Time elasped: 16.71855664253235
Epoch [32/100], Train Step [75/147], Train Loss = 2.11515
Time elasped: 25.14756155014038
Epoch [32/100], Train Step [100/147], Train Loss = 2.12813
Time elasped: 33.587156772613525
Epoch [32/100], Train Step [125/147], Train Loss = 2.09728
Time elasped: 42.02021503448486
Test Step [25/36], Acc: 10.50000
Epoch [32/100] 	 Test Accuracy Over Test Dataset: 10.417
Epoch [33/100], Train Step [25/147], Train Loss = 2.07110
Time elasped: 8.26327705

Epoch [46/100], Train Step [100/147], Train Loss = 2.08676
Time elasped: 33.432398080825806
Epoch [46/100], Train Step [125/147], Train Loss = 2.05911
Time elasped: 41.86339259147644
Test Step [25/36], Acc: 39.00000
Epoch [46/100] 	 Test Accuracy Over Test Dataset: 37.153
Saving......
Epoch [47/100], Train Step [25/147], Train Loss = 2.08339
Time elasped: 8.260705947875977
Epoch [47/100], Train Step [50/147], Train Loss = 2.15352
Time elasped: 16.667266607284546
Epoch [47/100], Train Step [75/147], Train Loss = 2.09937
Time elasped: 25.073714017868042
Epoch [47/100], Train Step [100/147], Train Loss = 2.00031
Time elasped: 33.473856925964355
Epoch [47/100], Train Step [125/147], Train Loss = 2.00955
Time elasped: 41.88216304779053
Test Step [25/36], Acc: 16.50000
Epoch [47/100] 	 Test Accuracy Over Test Dataset: 16.667
Epoch [48/100], Train Step [25/147], Train Loss = 2.07075
Time elasped: 8.243251085281372
Epoch [48/100], Train Step [50/147], Train Loss = 2.10790
Time elasped: 16.6496

Epoch [61/100], Train Step [125/147], Train Loss = 1.99172
Time elasped: 41.90714716911316
Test Step [25/36], Acc: 36.50000
Epoch [61/100] 	 Test Accuracy Over Test Dataset: 35.069
Saving......
Epoch [62/100], Train Step [25/147], Train Loss = 2.01849
Time elasped: 8.250710487365723
Epoch [62/100], Train Step [50/147], Train Loss = 2.03069
Time elasped: 16.666144847869873
Epoch [62/100], Train Step [75/147], Train Loss = 2.08219
Time elasped: 25.065954208374023
Epoch [62/100], Train Step [100/147], Train Loss = 2.03558
Time elasped: 33.489585399627686
Epoch [62/100], Train Step [125/147], Train Loss = 2.03281
Time elasped: 41.91687297821045
Test Step [25/36], Acc: 24.50000
Epoch [62/100] 	 Test Accuracy Over Test Dataset: 22.917
Epoch [63/100], Train Step [25/147], Train Loss = 2.02194
Time elasped: 8.246350049972534
Epoch [63/100], Train Step [50/147], Train Loss = 2.01065
Time elasped: 16.67276883125305
Epoch [63/100], Train Step [75/147], Train Loss = 2.00092
Time elasped: 25.080274

Test Step [25/36], Acc: 10.00000
Epoch [76/100] 	 Test Accuracy Over Test Dataset: 8.333
Saving......
Epoch [77/100], Train Step [25/147], Train Loss = 2.03423
Time elasped: 8.272079229354858
Epoch [77/100], Train Step [50/147], Train Loss = 2.04208
Time elasped: 16.709893703460693
Epoch [77/100], Train Step [75/147], Train Loss = 2.00021
Time elasped: 25.14108943939209
Epoch [77/100], Train Step [100/147], Train Loss = 2.06836
Time elasped: 33.56552982330322
Epoch [77/100], Train Step [125/147], Train Loss = 2.03045
Time elasped: 41.99404048919678
Test Step [25/36], Acc: 20.50000
Epoch [77/100] 	 Test Accuracy Over Test Dataset: 21.181
Epoch [78/100], Train Step [25/147], Train Loss = 2.03790
Time elasped: 8.260721683502197
Epoch [78/100], Train Step [50/147], Train Loss = 2.00715
Time elasped: 16.687641620635986
Epoch [78/100], Train Step [75/147], Train Loss = 2.00337
Time elasped: 25.13334369659424
Epoch [78/100], Train Step [100/147], Train Loss = 2.03055
Time elasped: 33.57151985

Epoch [91/100] 	 Test Accuracy Over Test Dataset: 16.667
Saving......
Epoch [92/100], Train Step [25/147], Train Loss = 1.97878
Time elasped: 8.250142812728882
Epoch [92/100], Train Step [50/147], Train Loss = 1.98362
Time elasped: 16.668227910995483
Epoch [92/100], Train Step [75/147], Train Loss = 2.02872
Time elasped: 25.075679540634155
Epoch [92/100], Train Step [100/147], Train Loss = 1.96995
Time elasped: 33.4908881187439
Epoch [92/100], Train Step [125/147], Train Loss = 1.96705
Time elasped: 41.90756297111511
Test Step [25/36], Acc: 18.00000
Epoch [92/100] 	 Test Accuracy Over Test Dataset: 16.667
Epoch [93/100], Train Step [25/147], Train Loss = 1.97408
Time elasped: 8.241830110549927
Epoch [93/100], Train Step [50/147], Train Loss = 2.00534
Time elasped: 16.6566321849823
Epoch [93/100], Train Step [75/147], Train Loss = 1.98957
Time elasped: 25.06434154510498
Epoch [93/100], Train Step [100/147], Train Loss = 1.98386
Time elasped: 33.47387099266052
Epoch [93/100], Train Step 